# IBM Tech 2025 Lab


## LAB B: RHELAI & InstructLab


This python notebook is a simple chatbot that uses the requests package to `chat` with a model(s) served on a RHELAI VSI Instance using InstructLab.

If you are unfamiliar with Jupyter Notebook interface, you can quickly go through [documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/notebook-editor.html?context=wx&locale=en&audience=wdp)

### To use this Notebook:
Run each cell below one by one and make sure it completes successfully.

When prompted enter your query and hit Enter. Type 'exit' or 'quit' to terminate the chat loop and then manually select and continue running from next cell

Some example questions are provided.

You may run/repeat the query cells by entering and trying different queries

### Goals

1) Chat with a model served on a RHELAI VSI using InstructLab VLLM
2) Try a document summarization usecase using external assets
3) Chat to expose some limitations of base granite model (*ibm-granite/granite-3.1-8b-base*)
4) Chat with a fine-tuned model that has been trained on synthetic generated data (SGD)

### LAB B - Exercise 1.a: 
Below is an outline of the code. The code is set up in Python Notebook cells.

1. Set up required libraries
2. Set the variables for accessing the model server.
3. Define the function to chat with model
4. Chat with the model

### LAB B - Exercise 1.b:
1. Load your external data as assets
2. Insert code snippet to read data
3. Chat with the Model

### LAB B - Exercise 2 & 3
1. Review the Model Server URL
2. Chat with the Model

#### 1. Install Pre-requisite libraries

In [1]:
!pip install requests

In [2]:
import requests
import json
import getpass

#### 2. Set the variables for accessing the model server

In [3]:
# Define the model server URL (ensure it's correct for your deployment)
MODEL_SERVER_URL = "https://ilab-all.industry-cloud-demo.ibm.com/v1/chat/completions"

# Securely get API key or authentication token if required
API_KEY = "popcorn-applies-flourish"

#### 3. Define function to chat with model using request package

In [4]:
def chat_with_model(prompt):
    headers = {"Content-Type": "application/json"}
    if API_KEY:
        headers["Authorization"] = f"Bearer {API_KEY}"
    
    payload = {
    "model": "rhelai_service_model",
    "messages": [
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt}
    ],
    "stop": ["<|"]
    }
    
    try:
        response = requests.post(MODEL_SERVER_URL, headers=headers, json=payload, verify=True)
        response.raise_for_status()
        data = response.json()
        return data.get("choices", [{}])[0].get("message", {}).get("content", "No response received.")
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return f"Error communicating with model: {e}"


#### 4. Provide a prompt to chat with the served model. Wait for response. Type 'exit' or 'quit' to terminate the chat loop and manually move to next cell.

   Some sample prompts are
   1) Hello, What can you do for me?
   2) What are some of the good places to visit in Singapore?
   3) What is the best way to travel around Singapore?
   4) How hot does it get in Singapore during end of March and should i carry an umbrella?

In [5]:
# Chat loop
while True:
    user_input = input("You: Please input a prompt and hit enter")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    elif user_input.strip() == "":
        continue
    
    print(f"Please wait....")
    response = chat_with_model(user_input)
    print(f"Model: {response}")

You: Please input a prompt and hit enter What can you do for me?


Please wait....
Model: I can provide answers to your questions and help you with tasks.



You: Please input a prompt and hit enter What are some of the must see places in Singapore?


Please wait....
Model: Singapore is a vibrant city-state known for its rich cultural heritage, iconic landmarks, and modern attractions. Here are some of the must-see places in Singapore:

1. **Marina Bay Sands**: This luxurious resort complex is a must-visit for its stunning architecture, including a 150-meter-high infinity pool, and offers panoramic views of the city.

2. **Gardens by the Bay**: This lush, man-made garden features a variety of attractions, including the Supertree Grove, a Cloud Forest, and the Flowers Dome and Glasshouse.

3. **Chinatown**: This historic neighborhood is a must-visit for its vibrant street life, traditional architecture, and numerous temples, such as the Sri Mariamman Temple.

4. **Little India**: This neighborhood is a must-visit for its colorful and bustling atmosphere, with numerous shops, restaurants, and temples, such as the Sri Veeramakaliamman Temple.

5. **Raffles Hotel**: This iconic hotel is a must-visit for its rich history, luxurious accom

You: Please input a prompt and hit enter What is the best way to travel around Singapore?


Please wait....
Model: What kind of information are you looking for?



You: Please input a prompt and hit enter What is the best way to travel around Singapore?


Please wait....
Model: There are several ways to travel around Singapore, including the MRT (Mass Rapid Transit) system, buses, taxis, and car rentals. The MRT is the most convenient and efficient way to travel within the city, while buses and taxis are also widely available. Car rentals can be a good option if you plan to travel outside of the city or if you prefer to have your own transportation.



You: Please input a prompt and hit enter exit


Exiting chat.


### Exercise 1.b: Summarize documents

Load your external data as assets and use them in your queries. Here we will load the document into associated COS bucket first and then use its contents to form prompts.

You have two options

1. Use the existing code by copy-pasting the below code block into new cell below

```
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='pQT19GjBZoU33LeoUNiT-dAQf8osQUYpHdzs5lWwNQIR',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'rhelaiinstructlab-donotdelete-pr-tjhwbcblh1zk4y'
object_key = 'RES0217.txt'

# load data of type "text/plain" into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
```

*OR*

2. Follow the instructions at https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/load-and-access-data.html?context=wx&locale=en&audience=wdp to upload new data and insert code to read it.
   1) From the toolbar, click the Upload asset to project icon (![Image](https://www.ibm.com/docs/en/SSQNUZ_5.1.x/wsj/console/images/find_data_icon.png)) and add your file (Text files upto 5KB).
   2) Click the Code snippets icon ![Image](https://www.ibm.com/docs/en/SSQNUZ_5.1.x/wsj/analyze-data/images/code-snippets-icon.png), click _Read data_, and then _"Select data from your project"_, under _Data Asset_ category, find the file uploaded.
   3) From the _Load as_ drop-down list, select the load option that you prefer(StreamingBody object). If you select Credentials, only file access credentials will be generated.
   4) Click in an empty code cell in your notebook and then click _Insert code to cell_ to insert the generated code. Alternatively, click to _copy_ the generated code to the clipboard and then paste the code into your notebook.

#### 5. Insert code snippet to read data

In [6]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='pQT19GjBZoU33LeoUNiT-dAQf8osQUYpHdzs5lWwNQIR',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'rhelaiinstructlab-donotdelete-pr-tjhwbcblh1zk4y'
object_key = 'RES0217.txt'

# load data of type "text/plain" into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

#### 6. Review the document contents
Update the document = &lt;variable&gt;.read() statement below with the variable into which body of the object is available (from last line of previous cell)

In [7]:
document = streaming_body_1.read()
print(f"Document read is \n {document.decode('utf-8')}")

Document read is 
 D: So I understand you removed insects from your leg earlier today?

P: Yes I did, it was uh it was behind my left uh knee kind of in like the joint behind where it bends. Um it was stuck pretty good, it's still kind of red and tender there. I noticed it because the back of my knee was starting to just feel a bit sore, um not the joint itself, but just kind of the skin, the whole area back there. Um and I found this tick, I brought I brought it in with me if you wanted to have a look at it, um it's in like this little plastic container, uh and I don't really know when I got it. I was hiking on Friday um and then Saturday I was out in my garden, I hope they're not around my house, I hope that it was from hiking um and not gardening, but uh I did spend a lot of time outside on Saturday as well.

D: OK, so sounds like it's been a little bit of time since you were last outdoors to when you first noticed it?

P: Yeah.

D: OK, so when you picked it off the tick, obviously 

#### 7. Use the document read and prompt with it.
  
Sample Prompts:

1) This is a conversation between a doctor and patient, provide a diagnosis and plan the next steps. Explain your response.
2) This is a conversation between a doctor and patient, is there an emergency involved or does this situation need immediate medical attention?
3) Based on the above conversation between a doctor and patient, what are the major concerns discussed and course of treatment advised.

In [8]:
# Chat loop
while True:
    user_input = input("You: Input a prompt and hit Enter")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    elif user_input.strip() == "":
        continue
    
    print(f"Please wait....")
    response = chat_with_model(f"{document} \n {user_input}")
    print(f"Model: {response}")

You: Input a prompt and hit Enter This is a conversation between a doctor and patient, provide a diagnosis and plan the next steps. Explain your response.


Please wait....
Model: Diagnosis: The patient has experienced a tick bite, but it does not appear to be a current Lyme disease infection due to the lack of a rash and other symptoms. However, the patient should be evaluated for Lyme disease due to the possibility that the tick could have been carrying the bacteria.

Plan:
1. Send the tick to a local laboratory for species identification.
2. Send blood work to check for titres or serology that may indicate Lyme disease.
3. Start the patient on antibiotics as a prophylactic measure, given the length of time the tick has been on the patient's body.
4. Evaluate the patient's joints and other symptoms for any signs of Lyme disease.
5. Discuss the patient's cholesterol levels and medications with the family physician or doctor.

Explanation: The conversation between the doctor and patient indicates that the patient has experienced a tick bite, but it does not appear to be a current Lyme disease infection due to the lack of a rash and other s

You: Input a prompt and hit Enter exit


Exiting chat.


### Exercise 2: Ask about IBM Cloud for Financial Services

The base model is not trained on IBM Cloud for financial services and starts to hallucinate. Prompt to ask about FSCloud

Sample Prompts
1) What is IBM Cloud for Financial Services and what industry clients is it focussed at?
   > Expected: IBM Cloud for Financial Services is a first-of-its-kind cloud, designed to protect even your most sensitive data and AI workloads. With built-in security and controls informed by the industry, IBM Cloud for Financial Services can help you optimize your infrastructure and demonstrate compliance so you can focus on what matters most: delivering value for your clients.
   > IBM Cloud for Financial Services is focused on serving financial institutions, with a particular emphasis on those that deal with large amounts of sensitive data and complex workloads. This includes banks, insurance companies, investment firms, and other financial institutions that handle large volumes of financial transactions and data.

2) The security control requirements of IBM Cloud for Financial services are based on what?
   > Expected: The security control requirements of IBM Cloud for Financial Services are based on industry standards such as NIST 800-53 Rev 4 and cloud best practices.
   
3) Can ISV products and services be validated for IBM Cloud for Financial Services?
   > Expected: Yes, ISV products and services can be validated for IBM Cloud for Financial Services. This validation process is designed to ensure that the products and services meet the necessary security, compliance, and resiliency requirements for financial institutions.
   
Notice how the responses are vague and in some cases downright incorrect. The base model is hallucinating.

#### 8. Set the endpoint for VSI instance serving base model and chat with the model

In [9]:
# Define the base model server URL (ensure it's correct for your deployment)
MODEL_SERVER_URL = "https://ilab-base.industry-cloud-demo.ibm.com/v1/chat/completions"

In [ ]:
# Chat loop
while True:
    user_input = input("You: Please input a prompt and hit Enter")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    elif user_input.strip() == "":
        continue
    
    print(f"Please wait....")
    response = chat_with_model(f"{user_input}")
    print(f"Model: {response}")

You: Please input a prompt and hit Enter What is IBM Cloud for Financial Services?


Please wait....
Model: IBM Cloud for Financial Services is a secure and compliant environment for financial institutions to build, deploy, and manage applications in the cloud. It is designed to meet the stringent regulatory requirements of the financial services industry. It provides a range of security and compliance features, including data encryption, access controls, and audit trails. It also includes pre-integrated services for common financial services workflows, such as fraud detection and anti-money laundering.



You: Please input a prompt and hit Enter What are the control requirements for IBM Cloud for Financial Services based on?


Please wait....
Model: The control requirements for IBM Cloud for Financial Services are based on industry regulations such as the New York State Department of Financial Services IT Examination Program Requirements for Cloud Computing Services (2015) and the Multi-State Information Sharing and Analysis Center (MS-ISAC) and Financial Services - Information Sharing and Analysis Center (FS-ISAC) jointly published the "Cloud Computing Profile" (2011). These resources provide a comprehensive set of requirements for financial institutions to consider when evaluating cloud service providers.



### Exercise 3: Fine-tuned model

The base model is fine-tuned on a knowledge base of IBM Cloud for Financial Services. The fine-tuned model is served on a different endpoint. 

Repeat the prompts to learn about IBM Cloud for Financial Services.

In [ ]:
#### 9. Set the endpoint for VSI instance serving the fine tuned model and chat with the model

In [ ]:
# Define the base model server URL (ensure it's correct for your deployment)
MODEL_SERVER_URL = "https://ilab-tuned.industry-cloud-demo.ibm.com/v1/chat/completions"

In [ ]:
# Chat loop
while True:
    user_input = input("You: Please input a prompt and hit Enter")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    elif user_input.strip() == "":
        continue
    
    print(f"Please wait....")
    response = chat_with_model(f"{user_input}")
    print(f"Model: {response}")